# Automated Anomaly Detection
**Objective**: Understand and practice automated anomaly detection using various techniques.

**Task**: Autoencoders for Anomaly Detection

**Steps**:
1. Data Set: Download a dataset of electricity consumption data.
2. Build an Autoencoder: Construct a simple autoencoder using a neural network for the
normal consumption data.
3. Identify Anomalies: Use the trained model to reconstruct the data and identify anomalies based on reconstruction error.
4. Visualize: Plot both the actual and reconstructed data to highlight anomalies.

In [ ]:
# write your code from here


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Simulate sample electricity consumption data
np.random.seed(42)
normal_data = np.random.normal(loc=0.5, scale=0.1, size=500)  # Normal consumption
anomalies = np.array([1.2, 1.3, 0.05, 1.4, 0.02])  # Injected anomalies
consumption_data = np.concatenate([normal_data, anomalies])
df = pd.DataFrame({'Consumption': consumption_data})

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df[['Consumption']])

# Step 2: Build a simple autoencoder
model = Sequential([
    Dense(8, activation='relu', input_shape=(1,)),
    Dense(4, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(data_scaled, data_scaled, epochs=50, batch_size=32, verbose=0)

# Step 3: Use reconstruction error to detect anomalies
reconstructions = model.predict(data_scaled)
reconstruction_error = np.mean(np.square(data_scaled - reconstructions), axis=1)

# Set threshold: Mean + 3*std of error from first 500 normal samples
threshold = np.mean(reconstruction_error[:500]) + 3 * np.std(reconstruction_error[:500])
df['ReconstructionError'] = reconstruction_error
df['Anomaly'] = df['ReconstructionError'] > threshold

# Step 4: Visualization
plt.figure(figsize=(14, 6))
plt.plot(df['Consumption'], label='Actual Consumption')
plt.plot(scaler.inverse_transform(reconstructions), label='Reconstructed', alpha=0.7)
plt.scatter(df.index[df['Anomaly']], df['Consumption'][df['Anomaly']], color='red', label='Anomalies')
plt.title("Electricity Consumption with Anomalies Detected by Autoencoder")
plt.xlabel("Time Index")
plt.ylabel("Consumption")
plt.legend()
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'tensorflow'